# Title.ipynb
## Generates Title from Fetched Data
### LSTM Code by https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

# Imports

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM

# Global Variables

In [2]:
FILE_LOCATION = "https://raw.githubusercontent.com/alexandre-lavoie/youtube-bot/master/data/database/CA_viewCount.csv"
MIN_TITLE_LENGTH = 11
NUMBER_OF_TITLES = 10
EPOCHS = 20
BATCH_SIZE = 128

# Read and Cleanup Database

In [3]:
video_database = pd.read_csv(FILE_LOCATION)

In [4]:
cleanup_database = video_database.replace('[^\x00-\x7F]+','',regex=True)

# Get Character Dictionary

In [5]:
chars = [chr(i) for i in range(ord('!'), ord('Z')+1)]
chars.extend(['|', ' ', '[', ']', '_'])
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
n_vocabs = len(chars)

# Converts Titles Into Training Data

In [6]:
title_int = []
for title in cleanup_database["title"]:
    title_int.append ([char_to_int[letter] for letter in title.upper()])

In [7]:
dataX = []
dataY = []

for title in title_int:
    if(len(title) > MIN_TITLE_LENGTH):
        for i in range(0, len(title)-MIN_TITLE_LENGTH):
            dataX.append(title[i:(i+MIN_TITLE_LENGTH)])
            dataY.append(title[i+MIN_TITLE_LENGTH])
        
n_patterns = len(dataX)

In [8]:
print("Vocab Length: " + str(n_vocabs))
print("Pattern Length: " + str(MIN_TITLE_LENGTH))
print("Number of patterns: " + str(n_patterns))

Vocab Length: 63
Pattern Length: 11
Number of patterns: 19449


In [9]:
X = np.reshape(dataX, (n_patterns, MIN_TITLE_LENGTH, 1))
X = X/float(n_vocabs)
y = np.eye(len(chars))[dataY]

# LSTM

In [10]:
model = Sequential()
model.add(CuDNNLSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Training

In [11]:
model.fit(X, y, epochs=EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/20
19449/19449 [==============================] - 9s 478us/step - loss: 3.2653
Epoch 2/20
19449/19449 [==============================] - 3s 170us/step - loss: 3.2027
Epoch 3/20
19449/19449 [==============================] - 3s 170us/step - loss: 3.1962
Epoch 4/20
19449/19449 [==============================] - 3s 170us/step - loss: 3.1815
Epoch 5/20
19449/19449 [==============================] - 3s 171us/step - loss: 3.1348
Epoch 6/20
19449/19449 [==============================] - 3s 171us/step - loss: 3.0785
Epoch 7/20
19449/19449 [==============================] - 3s 171us/step - loss: 3.0212
Epoch 8/20
19449/19449 [==============================] - 3s 170us/step - loss: 2.9751
Epoch 9/20
19449/19449 [==============================] - 3s 171us/step - loss: 2.9343
Epoch 10/20
19449/19449 [==============================] - 3s 171us/step - loss: 2.8934
Epoch 11/20
19449/19449 [==============================] - 3s 170us/step - loss: 2.8557 0s - loss: 2
Epoch 12/20
19449/19449 [===

# Predictions

In [12]:
for _ in range(NUMBER_OF_TITLES):
    start = np.random.randint(0, len(dataX)-1)
    patt = dataX[start]
    text = ""
    for value in patt:
        text += int_to_char[value] 

    for i in range(100):
        xx = np.reshape(patt, (1, len(patt), 1))
        xx = xx / float(n_vocabs)
        prediction = model.predict(xx, verbose=0)
        index = np.argmax(prediction)
        text += int_to_char[index]
        patt.append(index)
        patt = patt[1:len(patt)]

    print(text)

BELIEVE IT THE WOUREE VIDEO) FT. DAAT   OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE 
TO WIN NINT TO BOO TOEE SIDEO) FT. DAAT   OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO TH
NCE STEWART TO BOO TOEE SIDEO) FT. DAAT   OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO TH
IG ENGLISH TOOE FO THE WOAEE TOU COO TOEE SIDEO) FT. DAAT   OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO 
RIS CLUB SCE WOAEO   OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO
ON SHARK AT THE WOAEE TOU COO TOEE SIDEO) FT. DAAT   OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO
FICIAL TRAILER - TIE OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO
REVENGE MAKE TOOES - TOE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  OO TH
(COLLEGE DENE TOACT OO COO THE WOUREE VIDEO) FT. DAAT   OO THE  OO THE  OO THE  OO THE  OO THE  OO THE  

In [13]:
model.save("title.h5")